# Data generation (Training space)  

The training space, which includes the design space and given moment capacity, is established here. The design space is built upon a 3D mesh grid equally discretized across the width, height, and reinforcement ratio dimensions, with the discretization step denoted as 'n' and spanning the defined range for each component of the design space.  The resulting mesh grid forms an n*3 matrix. The moment capacity linked to each node on this mesh grid is then calculated analytically, resulting in an n*1 matrix. Both the design space (output of NN) and moment capacity (input of NN) are saved in .npy data files for further utilization.

In [6]:
from pathlib import Path
import numpy as np
from bmcs_cross_section.api import AnaFRPBending
from bmcs_cross_section.api import ACI440

home_dir = Path().home()
ml_data = home_dir / 'ml_data'
if not ml_data.exists():
    ml_data.mkdir()  
    
    

In [9]:
rho_range = np.linspace(0.01, 2, 301)  # Unit: %,  steps:0.01 %
b_range = np.linspace(10, 1000, 100)  # unit: mm, steps: 10 mm %, should not start from 0 to avoid division by zero
d_range = np.linspace(10, 1000, 100)  # unit: mm, steps:10 mm %, should not start from 0 to avoid division by zero
rho_mesh, b_mesh, d_mesh = np.meshgrid(rho_range, b_range, d_range, sparse=False)

@np.vectorize
def M_u_Ana_TTC(rho, b, d): # full strength utilization
    A_f = rho * b * d/100
    M_u = AnaFRPBending(A_f=A_f, f_cm=50, E_f=70000, f_fu=900, b=b, d=d).M_u
    return M_u

@np.vectorize
def Mn_ACI440 (rho,b,d):
    A_f = rho * b * d /100
    Mn = ACI440.get_M_n(A_f=A_f, f_fu=900, E_f=70000, f_c=50, b=b, d=d)
    # f_c [MPa]: compressive strength of concrete (typically specified compressive strength f'c = f_ck)
    # f_fu [MPa]: tensile strength of FRP reinf.
    # E_f [MPa]: e-modulus of FRP reinf.
    # Balanced FRP reinforcement ratio
    return Mn

# Calculate the mu_values
Mn = Mn_ACI440(rho_mesh, b_mesh, d_mesh)
M_u = M_u_Ana_TTC(rho_mesh, b_mesh, d_mesh)

# Save the meshgrid arrays and Mn_values
np.save(ml_data / 'ACI440_Mn.npy', Mn)
np.save(ml_data / 'Ana_TTC_M_u.npy', M_u)
np.savez(ml_data / 'designspace_meshgrid_data.npz', rho_mesh=rho_mesh, b_mesh=b_mesh, d_mesh=d_mesh)


# 2m 40.8s 

Size of b_mesh: (100, 301, 100)
Size of d_mesh: (100, 301, 100)
Size of p_mesh: (100, 301, 100)
Size of mu_values: (100, 301, 100)


In [4]:
rho_range = np.linspace(0.01, 2, 301)  # Unit: %,  steps:0.01 %
b_range = np.linspace(10, 1000, 100)  # unit: mm, steps: 10 mm %, should not start from 0 to avoid division by zero
d_range = np.linspace(10, 1000, 100)  # unit: mm, steps:10 mm %, should not start from 0 to avoid division by zero

inputs = []
output = []

# Generate combinations and calculate M_u
for rho in rho_range:
    for b in b_range:
        for d in d_range:
            # Calculate A_f based on p, b, and d (Assuming a simple calculation)
            A_f = rho * b * d / 100

            # Initialize AnaFRPBending object
            M_u_TTC = AnaFRPBending(A_f=A_f, f_cm = 50, E_f = 70000, f_fu = 900, b=b, d=d).M_u
                      
            # Append inputs and output to the lists
            inputs.append([rho, b, d])
            output.append(M_u_TTC)

# Convert lists to numpy arrays
inputs = np.array(inputs)
output = np.array(output)
np.save(ml_data / 'Design_space2.npy', inputs)
np.save(ml_data / 'Mu2.npy', output)

